# Giorno 3 - Pratica: My First Claude App

## Obiettivi
- Setup ambiente e libreria Anthropic
- Prima chiamata API
- Esperimenti con parametri
- Costruire un chatbot CLI

**Durata:** 3h 30min

---
## Parte 0: Setup Ambiente (30 min)

### 0.1 Installazione Librerie

In [ ]:
# Installa le librerie necessarie
!pip install anthropic python-dotenv

### 0.2 Creazione Account API

1. Vai su [console.anthropic.com](https://console.anthropic.com)
2. Crea un account (o accedi)
3. Vai su "API Keys"
4. Clicca "Create Key"
5. Copia la chiave (inizia con `sk-ant-...`)

**IMPORTANTE:** Non condividere mai questa chiave!

### 0.3 Configurazione API Key

**Metodo 1: Variabile d'ambiente (consigliato)**

Nel terminale:
```bash
export ANTHROPIC_API_KEY="sk-ant-..."
```

**Metodo 2: File .env (alternativa)**

Crea un file `.env` nella stessa cartella:
```
ANTHROPIC_API_KEY=sk-ant-...
```

In [ ]:
# Carica la API key
import os
from dotenv import load_dotenv

# Carica .env se presente
load_dotenv()

# Verifica che la key esista
api_key = os.getenv("ANTHROPIC_API_KEY")

if api_key:
    print(f"API Key trovata: {api_key[:10]}...")
else:
    print("ATTENZIONE: API Key non trovata!")
    print("Imposta ANTHROPIC_API_KEY come variabile d'ambiente")

In [ ]:
# Inizializza il client Anthropic
from anthropic import Anthropic

client = Anthropic()  # Usa automaticamente ANTHROPIC_API_KEY
print("Client Anthropic inizializzato con successo!")

---
## Parte 1: Prima Chiamata API (30 min)

### 1.1 Hello Claude!

In [ ]:
# La nostra prima chiamata API!
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Ciao Claude! Presentati in italiano."}
    ]
)

print(response.content[0].text)

### 1.2 Esploriamo la Risposta

In [ ]:
# Esaminiamo l'oggetto response
print("=== STRUTTURA RISPOSTA ===")
print(f"ID: {response.id}")
print(f"Modello: {response.model}")
print(f"Ruolo: {response.role}")
print(f"Stop reason: {response.stop_reason}")
print(f"\n=== TOKEN ===")
print(f"Input tokens: {response.usage.input_tokens}")
print(f"Output tokens: {response.usage.output_tokens}")
print(f"Totale: {response.usage.input_tokens + response.usage.output_tokens}")

### 1.3 Esercizio: La Tua Prima Chiamata

Modifica il codice sotto per fare una domanda a tua scelta.

In [ ]:
# Scrivi la tua domanda
mia_domanda = "[SCRIVI QUI LA TUA DOMANDA]"

response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": mia_domanda}
    ]
)

print(f"Domanda: {mia_domanda}")
print(f"\nRisposta: {response.content[0].text}")
print(f"\n[Token: {response.usage.input_tokens} + {response.usage.output_tokens}]")

---
## Parte 2: Esperimenti con Parametri (45 min)

### 2.1 Confronto Modelli

In [ ]:
# Stesso prompt su modelli diversi
prompt = "Spiega la fotosintesi in una frase."

models = [
    "claude-3-haiku-20240307",
    "claude-3-sonnet-20240229",
    # "claude-3-opus-20240229"  # Decommentare se vuoi testare (più costoso)
]

for model in models:
    print(f"\n=== {model} ===")
    response = client.messages.create(
        model=model,
        max_tokens=256,
        messages=[{"role": "user", "content": prompt}]
    )
    print(response.content[0].text)

**Osservazioni:**
- Quale modello ha dato la risposta migliore?
- Hai notato differenze di stile?

_[Scrivi qui]_

### 2.2 Esperimenti con Temperature

In [ ]:
# Stesso prompt con temperature diverse
prompt = "Inventa un nome per una startup di AI."

temperatures = [0.0, 0.5, 1.0, 1.5]

for temp in temperatures:
    print(f"\n=== Temperature: {temp} ===")
    # Facciamo 3 tentativi per vedere la variabilità
    for i in range(3):
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=50,
            temperature=temp,
            messages=[{"role": "user", "content": prompt}]
        )
        print(f"  {i+1}. {response.content[0].text.strip()[:50]}")

**Osservazioni:**
- Temperature 0.0: risposte ripetitive o diverse?
- Temperature alta: più creatività?

_[Scrivi qui]_

### 2.3 Effetto di max_tokens

In [ ]:
# Vedere l'effetto del limite token
prompt = "Racconta la storia di Cappuccetto Rosso."

for max_tok in [50, 150, 500]:
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=max_tok,
        messages=[{"role": "user", "content": prompt}]
    )
    print(f"\n=== max_tokens: {max_tok} (usati: {response.usage.output_tokens}) ===")
    print(f"Stop reason: {response.stop_reason}")
    print(response.content[0].text[:200] + "..." if len(response.content[0].text) > 200 else response.content[0].text)

### 2.4 System Prompt

In [ ]:
# Stesso prompt, system prompt diversi
user_prompt = "Cos'è Python?"

system_prompts = [
    "Rispondi come un professore universitario formale.",
    "Rispondi come uno YouTuber entusiasta.",
    "Rispondi in massimo 10 parole.",
    "Rispondi usando solo emoji e una parola."
]

for system in system_prompts:
    print(f"\n=== System: {system[:40]}... ===")
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=256,
        system=system,
        messages=[{"role": "user", "content": user_prompt}]
    )
    print(response.content[0].text)

---
## Parte 3: Gestione Conversazioni Multi-turn (45 min)

### 3.1 Problema: Claude Non Ricorda

In [ ]:
# Chiamata 1
response1 = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=256,
    messages=[{"role": "user", "content": "Mi chiamo Marco. Ricordalo!"}]
)
print("Risposta 1:", response1.content[0].text)

# Chiamata 2 (separata)
response2 = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=256,
    messages=[{"role": "user", "content": "Come mi chiamo?"}]
)
print("\nRisposta 2:", response2.content[0].text)

### 3.2 Soluzione: Passare la Cronologia

In [ ]:
# Chiamata 1
messages = [{"role": "user", "content": "Mi chiamo Marco. Ricordalo!"}]

response1 = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=256,
    messages=messages
)
print("Risposta 1:", response1.content[0].text)

# Aggiungi la risposta alla cronologia
messages.append({"role": "assistant", "content": response1.content[0].text})

# Chiamata 2 (con cronologia)
messages.append({"role": "user", "content": "Come mi chiamo?"})

response2 = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=256,
    messages=messages
)
print("\nRisposta 2:", response2.content[0].text)

### 3.3 Classe ChatSession

In [ ]:
class ChatSession:
    """
    Gestisce una sessione di chat con Claude.
    Mantiene la cronologia dei messaggi.
    """

    def __init__(self, system_prompt="Sei un assistente utile."):
        self.client = Anthropic()
        self.system = system_prompt
        self.messages = []

    def send(self, user_message):
        """Invia un messaggio e ottieni la risposta."""
        # Aggiungi messaggio utente
        self.messages.append({
            "role": "user",
            "content": user_message
        })

        # Chiama API
        response = self.client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1024,
            system=self.system,
            messages=self.messages
        )

        # Estrai risposta
        assistant_message = response.content[0].text

        # Salva nella cronologia
        self.messages.append({
            "role": "assistant",
            "content": assistant_message
        })

        return assistant_message, response.usage

    def clear(self):
        """Pulisce la cronologia."""
        self.messages = []

    def get_history(self):
        """Restituisce la cronologia."""
        return self.messages

In [ ]:
# Test della classe
chat = ChatSession("Sei un tutor di Python paziente e incoraggiante.")

risposta, usage = chat.send("Ciao! Voglio imparare Python.")
print(f"Claude: {risposta}\n")

risposta, usage = chat.send("Da dove comincio?")
print(f"Claude: {risposta}\n")

risposta, usage = chat.send("Cos'è una variabile?")
print(f"Claude: {risposta}")

In [ ]:
# Vediamo la cronologia
print("=== CRONOLOGIA ===")
for msg in chat.get_history():
    print(f"\n[{msg['role'].upper()}]")
    print(msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content'])

---
## Parte 4: Chatbot CLI Completo (45 min)

### 4.1 Chatbot Base

In [ ]:
def simple_chatbot():
    """Chatbot CLI semplice."""
    print("="*50)
    print("CHATBOT CLAUDE")
    print("Scrivi 'quit' per uscire")
    print("="*50)

    chat = ChatSession("Sei un assistente amichevole che risponde in italiano.")

    while True:
        user_input = input("\nTu: ").strip()

        if user_input.lower() == 'quit':
            print("\nArrivederci!")
            break

        if not user_input:
            continue

        try:
            risposta, usage = chat.send(user_input)
            print(f"\nClaude: {risposta}")
            print(f"\n[Token: {usage.input_tokens} + {usage.output_tokens}]")
        except Exception as e:
            print(f"\nErrore: {e}")

# Esegui il chatbot
# simple_chatbot()  # Decommenta per eseguire

### 4.2 Chatbot con Funzionalità Avanzate

In [ ]:
class AdvancedChatbot:
    """
    Chatbot avanzato con comandi speciali.
    """

    def __init__(self):
        self.client = Anthropic()
        self.messages = []
        self.system = "Sei un assistente utile che risponde in italiano."
        self.model = "claude-3-haiku-20240307"
        self.total_tokens = 0

    def process_command(self, command):
        """Gestisce i comandi speciali."""
        cmd = command.lower().strip()

        if cmd == '/help':
            return """
Comandi disponibili:
  /help     - Mostra questo messaggio
  /clear    - Pulisce la cronologia
  /history  - Mostra la cronologia
  /tokens   - Mostra token totali usati
  /system   - Cambia system prompt
  /quit     - Esci dal chatbot
"""

        elif cmd == '/clear':
            self.messages = []
            return "Cronologia cancellata."

        elif cmd == '/history':
            if not self.messages:
                return "Nessun messaggio nella cronologia."
            result = "\n=== CRONOLOGIA ===\n"
            for msg in self.messages:
                role = "Tu" if msg['role'] == 'user' else "Claude"
                text = msg['content'][:50] + "..." if len(msg['content']) > 50 else msg['content']
                result += f"{role}: {text}\n"
            return result

        elif cmd == '/tokens':
            return f"Token totali usati: {self.total_tokens}"

        elif cmd.startswith('/system '):
            self.system = cmd[8:]
            return f"System prompt aggiornato: {self.system}"

        return None  # Non è un comando

    def chat(self, user_message):
        """Invia messaggio e ricevi risposta."""
        self.messages.append({"role": "user", "content": user_message})

        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            system=self.system,
            messages=self.messages
        )

        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})

        self.total_tokens += response.usage.input_tokens + response.usage.output_tokens

        return assistant_message

    def run(self):
        """Esegue il chatbot."""
        print("="*50)
        print("CHATBOT AVANZATO")
        print("Scrivi /help per i comandi")
        print("="*50)

        while True:
            user_input = input("\nTu: ").strip()

            if not user_input:
                continue

            # Controlla comandi
            if user_input.startswith('/'):
                if user_input.lower() == '/quit':
                    print(f"\nArrivederci! Token totali: {self.total_tokens}")
                    break
                result = self.process_command(user_input)
                if result:
                    print(result)
                    continue

            # Chat normale
            try:
                risposta = self.chat(user_input)
                print(f"\nClaude: {risposta}")
            except Exception as e:
                print(f"\nErrore: {e}")
                self.messages.pop()  # Rimuovi messaggio fallito

# Esegui
# bot = AdvancedChatbot()
# bot.run()  # Decommenta per eseguire

---
## Parte 5: Salvataggio e Caricamento Conversazioni (30 min)

### 5.1 Salvare su File JSON

In [ ]:
import json
from datetime import datetime

class PersistentChatbot:
    """
    Chatbot che salva le conversazioni su file.
    """

    def __init__(self, save_file="chat_history.json"):
        self.client = Anthropic()
        self.save_file = save_file
        self.messages = []
        self.system = "Sei un assistente utile."
        self.load_history()

    def save_history(self):
        """Salva la cronologia su file."""
        data = {
            "system": self.system,
            "messages": self.messages,
            "last_updated": datetime.now().isoformat()
        }
        with open(self.save_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"Cronologia salvata in {self.save_file}")

    def load_history(self):
        """Carica la cronologia da file."""
        try:
            with open(self.save_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            self.system = data.get("system", self.system)
            self.messages = data.get("messages", [])
            print(f"Caricati {len(self.messages)} messaggi da {self.save_file}")
        except FileNotFoundError:
            print("Nessuna cronologia precedente trovata.")

    def chat(self, user_message):
        """Invia messaggio e salva."""
        self.messages.append({"role": "user", "content": user_message})

        response = self.client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1024,
            system=self.system,
            messages=self.messages
        )

        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})

        self.save_history()  # Salva dopo ogni messaggio

        return assistant_message

In [ ]:
# Test del chatbot persistente
pbot = PersistentChatbot("test_chat.json")

print(pbot.chat("Ciao! Oggi è una bella giornata."))
print("\n" + pbot.chat("Di cosa stavamo parlando?"))

In [ ]:
# Verifica il file salvato
with open("test_chat.json", 'r') as f:
    print(f.read())

---
## Esercizi Finali

### Esercizio 1: Chatbot Personalizzato

Crea un chatbot con un system prompt personalizzato per un caso d'uso specifico (es: tutor di matematica, consulente fitness, assistente cucina).

In [ ]:
# Il tuo chatbot personalizzato

system_prompt = """
[SCRIVI IL TUO SYSTEM PROMPT]
"""

chat = ChatSession(system_prompt)

# Test con alcune domande
print(chat.send("[LA TUA PRIMA DOMANDA]")[0])

### Esercizio 2: Batch Processing

Crea una funzione che prende una lista di domande e restituisce tutte le risposte.

In [ ]:
def batch_questions(questions, system="Rispondi in modo conciso."):
    """
    Elabora una lista di domande in batch.

    Args:
        questions: lista di stringhe
        system: system prompt da usare

    Returns:
        lista di tuple (domanda, risposta, token_usati)
    """
    # [IMPLEMENTA LA FUNZIONE]
    pass


# Test
domande = [
    "Qual è la capitale della Francia?",
    "Quanto fa 15 * 7?",
    "Chi ha scritto la Divina Commedia?"
]

# risultati = batch_questions(domande)
# for q, a, t in risultati:
#     print(f"D: {q}")
#     print(f"R: {a}")
#     print(f"Token: {t}\n")

---
## Conclusione

### Cosa abbiamo imparato:

1. **Setup API** - Configurazione client e API key
2. **Chiamate base** - Struttura richiesta/risposta
3. **Parametri** - model, max_tokens, temperature, system
4. **Multi-turn** - Gestire conversazioni con cronologia
5. **Persistenza** - Salvare e caricare conversazioni

### Prossimi passi:
Domani esploreremo **Claude Code** per lo sviluppo assistito da AI!